In [1]:
import torch
import pandas as pd
from sklearn.decomposition import PCA

import torch
import pandas as pd
from tqdm import tqdm
import sys
sys.path.append('/work/magroup/kaileyhu/synthetic_lethality/utils')

from extract_df_info import *
tqdm.pandas()

In [2]:
df = pd.read_hdf("/work/magroup/kaileyhu/res/gene_embeddings/geneformer.csv", "table")

In [3]:
dims = 128

In [4]:
pca = PCA(n_components=dims)

principalComponents = pca.fit_transform(df)

In [5]:
pca_df = pd.DataFrame(data = principalComponents, columns = [f"pc {i}" for i in range(dims)])

In [9]:
pca_df.index = df.index

In [10]:
pca_df.to_csv(f"/work/magroup/kaileyhu/res/gene_embeddings/geneformer_pca_{dims}.csv")

In [11]:
path = "/work/magroup/kaileyhu/res/perturbed/gf_12L_30M_i2048_SL/ESM_df"
esm_df = pd.read_hdf(f"{path}/ESM_128_emb_mat.h5", "table")

In [15]:
esm_df['gene'] = list(map(get_genes_from_index, list(esm_df.index)))
esm_df['gene'] = esm_df['gene'].apply(lambda x : x[5:])
gene_cols = [f"gene {i}" for i in range(dims)]

In [16]:
temp = esm_df['gene'].progress_apply(lambda x : pd.Series(list(pca_df.loc[x])) if x in pca_df.index else pd.Series([-100.0 for _ in range(dims)]))

100%|██████████| 2053880/2053880 [10:04<00:00, 3397.26it/s]


In [17]:
esm_df[gene_cols] = temp

In [18]:
print(esm_df.shape)
esm_df = esm_df[esm_df['gene 112'] != -100]
esm_df.drop(columns=['gene'], inplace = True)
print(esm_df.shape)

(2053880, 642)
(2040847, 641)


In [ ]:
path = "/work/magroup/kaileyhu/res/perturbed/gf_12L_30M_i2048_SL/geneformer_df"

esm_df.to_hdf(f"{path}/geneformer_{dims}_emb_mat.h5", key = "table")

In [20]:
_

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
ENSG00000000003,-0.023651,0.001416,0.023693,-0.001688,-0.000063,0.055200,-0.033565,0.001835,0.002906,0.050767,...,-0.067969,-0.046953,-0.062575,-0.010067,-0.194781,-0.034042,0.071299,-0.040417,-0.017178,0.152310
ENSG00000000419,0.019412,-0.061007,0.028338,-0.010694,-0.062602,0.111002,-0.007464,0.033390,0.028717,0.121114,...,0.009356,-0.083338,0.128770,0.146635,-0.104946,-0.122677,0.016736,-0.152884,0.054454,0.072711
ENSG00000000457,0.218134,-0.004801,0.043361,0.001295,-0.006084,0.067230,-0.057261,-0.035720,-0.008749,-0.000961,...,-0.135585,-0.064745,0.124235,0.003192,0.093108,-0.054768,0.095327,-0.125639,-0.078857,-0.215721
ENSG00000001036,-0.330152,-0.037557,0.032097,-0.003765,0.025903,0.098447,-0.003560,0.024460,-0.033782,0.098714,...,-0.056802,-0.073244,0.055809,0.092184,0.254563,-0.093984,0.027788,-0.114852,0.029564,0.015750
ENSG00000001084,0.069236,-0.048437,0.045485,-0.004585,-0.080787,0.070514,-0.023165,-0.007976,-0.001503,0.070342,...,-0.043399,-0.064771,-0.185735,0.036507,-0.020292,-0.080573,0.083737,-0.039322,-0.045695,-0.144979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000284194,-0.012658,-0.044564,0.048137,-0.000843,-0.070892,0.098326,-0.055570,-0.045259,-0.034218,0.017707,...,-0.040479,-0.076639,-0.226949,-0.002065,0.236911,-0.081732,0.070801,-0.061813,-0.070710,-0.176696
ENSG00000284770,0.081414,-0.020577,0.026313,0.003132,0.030907,0.063570,-0.028001,0.020523,0.096121,0.024628,...,-0.052129,-0.033462,0.016224,0.018434,0.128116,-0.047731,0.024031,-0.043192,-0.079192,-0.017510
ENSG00000288611,-0.126163,-0.061119,0.044752,-0.002434,-0.110060,0.101659,-0.052583,-0.086088,0.063138,0.111941,...,0.009301,-0.044578,0.113883,0.133582,0.067990,-0.057883,0.072924,-0.085499,-0.035412,0.000858
ENSG00000288825,0.022977,-0.013981,0.028981,0.002979,0.043080,0.069611,-0.023887,-0.051689,-0.019462,0.028811,...,-0.126583,-0.061228,-0.096709,0.020480,0.066392,-0.057076,-0.024781,-0.045141,-0.071638,-0.130556


In [22]:
esm_df

,0,1,2,3,4,5,6,7,8,9,...,gene 119,gene 120,gene 121,gene 122,gene 123,gene 124,gene 125,gene 126,gene 127,viability score
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
"('ACH-001289', 'gene_ENSG00000051341')",-0.005241,-0.005240,0.003060,0.000099,0.006640,0.001894,-0.006397,0.009086,-0.000369,-0.006278,...,-0.090203,0.118779,0.079299,-0.029972,0.140807,-0.133641,0.090890,0.002818,-0.059660,-0.380100
"('ACH-001538', 'gene_ENSG00000051341')",0.006738,0.010125,-0.005152,0.001353,-0.004012,-0.001398,0.003976,-0.009277,-0.006055,0.020137,...,-0.090203,0.118779,0.079299,-0.029972,0.140807,-0.133641,0.090890,0.002818,-0.059660,-0.659543
"('ACH-000233', 'gene_ENSG00000051341')",0.012463,0.001579,0.005601,-0.000914,0.004134,0.002549,-0.003315,-0.009647,-0.003803,0.007985,...,-0.090203,0.118779,0.079299,-0.029972,0.140807,-0.133641,0.090890,0.002818,-0.059660,-0.371839
"('ACH-000461', 'gene_ENSG00000051341')",-0.006240,-0.003171,0.006881,-0.002384,0.005634,-0.003116,0.014743,-0.002460,0.000401,0.009565,...,-0.090203,0.118779,0.079299,-0.029972,0.140807,-0.133641,0.090890,0.002818,-0.059660,-0.449324
"('ACH-001794', 'gene_ENSG00000051341')",-0.000864,0.000761,0.002266,-0.000997,0.002481,0.000293,-0.001737,0.001371,-0.003522,0.002755,...,-0.090203,0.118779,0.079299,-0.029972,0.140807,-0.133641,0.090890,0.002818,-0.059660,-0.571887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"('ACH-001128', 'gene_ENSG00000100926')",0.001396,-0.003924,-0.001502,-0.000176,-0.002986,-0.016980,0.007556,0.001213,-0.002266,0.024180,...,0.040559,-0.003923,0.000760,0.014655,0.086737,-0.044542,-0.010549,0.029690,0.020069,-0.148967
"('ACH-002669', 'gene_ENSG00000100926')",0.008073,-0.002751,-0.000530,0.001813,-0.007618,0.004425,0.002330,0.002746,0.002113,-0.011364,...,0.040559,-0.003923,0.000760,0.014655,0.086737,-0.044542,-0.010549,0.029690,0.020069,-0.206309
"('ACH-001858', 'gene_ENSG00000100926')",-0.000958,-0.000309,-0.000812,-0.002005,0.002194,0.008029,0.020479,-0.001229,0.010642,0.009958,...,0.040559,-0.003923,0.000760,0.014655,0.086737,-0.044542,-0.010549,0.029690,0.020069,-0.136147
